In [14]:
import string
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer 

from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import ComplementNB

from sklearn.metrics import confusion_matrix

In [15]:
df = pd.read_csv("riasec_agrea.csv")
df.head()

,gender,text,agrea_q
0,1,Study the structure of the human body\nStudy a...,2
1,1,Lay brick or tile\nFix a broken faucet\nStudy ...,2
2,1,NaN,1
3,1,Lay brick or tile\nAssemble electronic parts\n...,2
4,1,Do volunteer work at a non-profit organization...,2


In [16]:
def fn_stemmer(x):
    stemmer = PorterStemmer()
    def fn(y): return " ".join([ stemmer.stem(word) for word in word_tokenize(y) ])
    fn_v = np.vectorize(fn)
    return fn_v(x)

In [17]:
preprocess_pipe = Pipeline([
    ("col_trans", ColumnTransformer([
         ("text", Pipeline([
             ("stemmer", FunctionTransformer(fn_stemmer)),
             ("tfidf_vectorizer", TfidfVectorizer(
                encoding = "utf-8",
                lowercase = True,
                stop_words =  nltk.corpus.stopwords.words('english') + (string.punctuation).split(),
                ngram_range = (1,1),
                max_features = None,
                use_idf = True,
            ))   
         ]), "text"),
    ],
    remainder = "passthrough"))
])

In [18]:
X = df.drop("agrea_q", axis = 1)
y = df["agrea_q"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [19]:
X_train_prepared = preprocess_pipe.fit_transform(X_train)
X_test_prepared =  preprocess_pipe.transform(X_test)

TypeError: expected string or bytes-like object

In [ ]:
#preprocess_pipe["col_trans"].transformers_[0][1]["tfidf_vectorizer"].get_feature_names()

In [ ]:
clf = ComplementNB()
clf.fit(X_train_prepared, y_train)
clf.score(X_test_prepared, y_test)

In [ ]:
confusion_matrix(y_test, clf.predict(X_test_prepared))